# 图像卷积

## 互相关运算

In [25]:
import torch
import torch.nn as nn
import d2l.torch as d2l

In [26]:
def corr2d(X, K):
    """二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h +1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [27]:
"""测试"""
X = torch.tensor([[0.0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0.0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 卷积层

In [28]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

conv2d = Conv2D((2, 2))
P = torch.tensor([[0.0, 1, 2], [3, 4, 5], [6, 7, 8]])
conv2d(P)

tensor([[ 4.5302,  6.2701],
        [ 9.7498, 11.4897]], grad_fn=<AddBackward0>)

## 图像中目标的边缘检测

In [29]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [30]:
K = torch.tensor([[1, -1]])

In [31]:
Y = corr2d(X, K)
Z = corr2d(X.t(), K)
Y, Z  # K只能检测垂直边缘，无法检测水平边缘

(tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]),
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]))

## 学习卷积核

In [49]:
# 构造一个卷积层，只有一个批量大小，一个输出通道和形状为(1, 2)的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

print(conv2d.weight.data.reshape((1, 2)))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss: {l.sum():.3f}')

tensor([[-0.4273,  0.1729]])
epoch 2, loss: 8.929
epoch 4, loss: 1.648
epoch 6, loss: 0.338
epoch 8, loss: 0.082
epoch 10, loss: 0.024


In [50]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9714, -0.9987]])